In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.cluster import KMeans
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import joblib

# =============================
# 🔹 Load Dataset
# =============================
file_path = 'dataset_sdn.csv'
df = pd.read_csv(file_path, encoding='latin1')

# Identify numeric and categorical features
numerical_features = [feature for feature in df.columns if df[feature].dtypes != 'O' and feature != "label"]
print("The number of numerical features is", len(numerical_features), "and they are : \n", numerical_features)
categorical_features = [feature for feature in df.columns if df[feature].dtypes == 'O']
print("The number of categorical features is", len(categorical_features), "and they are : \n", categorical_features)
target = 'label'

# Separate features and target variable
X = df[numerical_features + categorical_features]
y = df[target]

# Split into training & testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# =============================
# 🔹 Handle Missing Values
# =============================
# Impute missing values in numeric features
num_imputer = SimpleImputer(strategy='mean')
X_train_numeric = num_imputer.fit_transform(X_train[numerical_features])
X_test_numeric = num_imputer.transform(X_test[numerical_features])

# Impute missing values in categorical features
cat_imputer = SimpleImputer(strategy='most_frequent')
X_train_categorical = cat_imputer.fit_transform(X_train[categorical_features])
X_test_categorical = cat_imputer.transform(X_test[categorical_features])

# =============================
# 🔹 Standardize Numeric Features
# =============================
scaler = StandardScaler()
X_train_numeric_scaled = scaler.fit_transform(X_train_numeric)
X_test_numeric_scaled = scaler.transform(X_test_numeric)

# =============================
# 🔹 Encode Categorical Features
# =============================
vectorizer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
X_train_cat_encoded = vectorizer.fit_transform(X_train_categorical)
X_test_cat_encoded = vectorizer.transform(X_test_categorical)

# =============================
# 🔹 Combine Processed Features
# =============================
X_train_final = np.hstack((X_train_numeric_scaled, X_train_cat_encoded))
X_test_final = np.hstack((X_test_numeric_scaled, X_test_cat_encoded))

# Convert target to NumPy arrays
y_train = y_train.values
y_test = y_test.values

# =============================
# 🔹 Helper Function for Evaluation
# =============================
def evaluate_model(y_true, y_pred, model_name):
    print(f"Evaluation for {model_name}:")
    print(f"Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(classification_report(y_true, y_pred))
    conf_matrix = confusion_matrix(y_true, y_pred)
    TN, FP, FN, TP = conf_matrix.ravel()
    print(f"TP: {TP}, TN: {TN}, FP: {FP}, FN: {FN}")
    print("\n")


# =============================
# 🔹 K-Means (Unsupervised)
# =============================
# Note: K-Means is unsupervised, so we use it for clustering and compare clusters with labels
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(X_train_final)
y_pred_kmeans = kmeans.predict(X_test_final)

# Map cluster labels to binary labels (0 or 1)
#from sklearn.utils.linear_assignment_ import linear_assignment
from scipy.optimize import linear_sum_assignment as linear_assignment

def align_cluster_labels(y_true, y_pred):
    confusion = confusion_matrix(y_true, y_pred)
    indices = linear_assignment(-confusion)
    mapped_labels = np.zeros_like(y_pred)
    for i, j in indices:
        mapped_labels[y_pred == j] = i
    return mapped_labels

y_pred_kmeans_aligned = align_cluster_labels(y_test, y_pred_kmeans)
evaluate_model(y_test, y_pred_kmeans_aligned, "K-Means")

# Save K-Means model
joblib.dump(kmeans, 'Modells/KMeans_model.pkl')
joblib.dump(scaler, 'Modells/KMeans_scaler.pkl')
joblib.dump(vectorizer, 'Modells/KMeans_vectorizer.pkl')

print("KMeans model, scaler, and vectorizer saved.")

The number of numerical features is 19 and they are : 
 ['dt', 'switch', 'pktcount', 'bytecount', 'dur', 'dur_nsec', 'tot_dur', 'flows', 'packetins', 'pktperflow', 'byteperflow', 'pktrate', 'Pairflow', 'port_no', 'tx_bytes', 'rx_bytes', 'tx_kbps', 'rx_kbps', 'tot_kbps']
The number of categorical features is 3 and they are : 
 ['src', 'dst', 'Protocol']


C:\Users\hp\anaconda3\envs\ml_env\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Evaluation for K-Means:
Accuracy: 0.6091
              precision    recall  f1-score   support

           0       0.61      1.00      0.76     12712
           1       0.00      0.00      0.00      8157

    accuracy                           0.61     20869
   macro avg       0.30      0.50      0.38     20869
weighted avg       0.37      0.61      0.46     20869

TP: 0, TN: 12712, FP: 0, FN: 8157


KMeans model, scaler, and vectorizer saved.


C:\Users\hp\anaconda3\envs\ml_env\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\hp\anaconda3\envs\ml_env\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\hp\anaconda3\envs\ml_env\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
